# Sentiment Analysis

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import regex as re
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import random
from tqdm import tqdm
tqdm.pandas()

# preprocessing 
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect
from langdetect import DetectorFactory

# word cloud
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
pd.options.mode.chained_assignment = None


from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

import warnings
warnings.filterwarnings(action='ignore')    # to suppress future warnings etc

<a id="0"></a>
#### Contents:
* [1. Dataset](#1)
    * [1.1. Loading Data](#1.1)
    * [1.2. Data Preprocessing](#1.2)
    * [1.3. Dataset Visualization](#1.3)
* [2. Lexicon based models](#2)
    * [2.1. VADER !!TODO:DELETE BEFORE SUBMISSION](#2.1)
    * [2.2. Model](#2.2)
    * [2.3. Evaluation](#2.3)
* [3. TextBlob Model](#3)
* [Advanced. Emotions in data](#advanced)

<a id="1"></a>
[<font size="+2" ><b>1. Dataset Presentation</b></font><br>](#0)

It is wise to explore our dataset first

<a id="1.1"></a>
[<font size="+1" ><b>1.1. Loading Data</b></font><br>](#0)
Load necessary data.

In [ ]:
# Removing unused columns
df_outside = pd.read_csv('dataset_transformed.csv', encoding='latin')
df_crawl = pd.read_csv('compiled_tweets.csv')
df_outside = df_outside[["Text", "Subjectivity", "Polarity"]]
df_crawl = df_crawl[["Text", "Subjectivity", "Polarity"]]
df_crawl = df_crawl[df_crawl['Subjectivity'].notna()].copy()
df = pd.concat([df_crawl, df_outside]).reset_index(drop=True)

<a id="1.2"></a>
[<font size="+1" ><b>1.2. Data Preprocessing</b></font><br>](#0)

As twitter texts are relatively unclean compared with others, special cleaning are applied to remove emojis and links

In [ ]:

# set seed
DetectorFactory.seed = 0

def language_detection(x):
    text = x['body_text'].split(" ")
    
    lang = "en"
    try:
        if len(text) > 50:
            lang = detect(" ".join(text[:50]))
        elif len(text) > 0:
            lang = detect(" ".join(text[:len(text)]))
    except Exception as e:
        all_words = set(text)
        try:
            lang = detect(" ".join(all_words))
        except Exception as e:
            try:
                # let's try to label it through the abstract then
                lang = detect(x['abstract'])
            except Exception as e:
                lang = "unknown"
                pass
    return lang

In [ ]:
def clean_text(text):
    text = str(text)
    text = re.sub(r'[^a-zA-Z ]+', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text)
    text = re.sub(r'^RT[\s]+', '', text)
    # text = re.sub(r'pic.twitter\S+', ' ', text)
    text = re.sub(r'#', '', text)
    text = text.lower()

    return text

def decontracted(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"don't", "do not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"it\'s", "it is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)

    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\“", "", text)
    text = re.sub(r"\”", "", text)
    text = re.sub(r"\…", "", text)

    return text


def remove_punc(tweet):
    tweet =  tweet.translate(str.maketrans('', '', string.punctuation))
    tweet = ' '.join([word for word in tweet.split()])
    tweet = tweet.lower()
    
    return tweet

In [ ]:
df['Text'] = df['Text'].progress_apply(clean_text).apply(decontracted).apply(remove_punc)
df.head()

### Stemming 

In [ ]:
# nltk.download('punkt')
# nltk.download('stopwords')

def stem_text(x):
    stemmer = PorterStemmer()
    x = word_tokenize(x)
    stem = ''
    
    for i in x:
        stem += stemmer.stem(i) + ' '
        
    return stem

df['stemmed_text'] = df['Text'].progress_apply(stem_text)
df.head()

### Lemmatization

In [ ]:
# nltk.download('wordnet')
# nltk.download('omw-1.4')

def lemmatize_text(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in tweet.split()]) for tweet in corpus]

df['lemmatized_text'] = lemmatize_text(df['Text'])
df.head()

### Stopwords Removal

In [ ]:
stop = stopwords.words('english')
additional_stopwords = ["'s","...","'ve","``","''","'m",'--',"'ll","'d", 'u', 'b', 'c', 'd', 'x', 'xf', 'f', 'p', 'xb']
stop = set(stop + additional_stopwords)

def remove_stopwords(x):
    x = word_tokenize(x)
    store_words = ''
    
    for i in x:
        if i not in stop:
            store_words += i + ' '
            
    return store_words

In [ ]:
df['cleaned_text'] = df['Text'].progress_apply(remove_stopwords)
df['cleaned_stemmed_text'] = df['stemmed_text'].progress_apply(remove_stopwords)
df['cleaned_lemmatized_text'] = df['lemmatized_text'].progress_apply(remove_stopwords)

df.head()

<a id="1.3"></a>
[<font size="+1"><b>1.3. Data Visualization</b></font><br>](#0)

In [ ]:
print("========== number of words in the corpus ==========")

print(f"original: {df['Text'].apply(lambda x: len(x.split())).sum()}")

print(f"after stemming: {df['stemmed_text'].apply(lambda x: len(x.split())).sum()}")

print(f"after lemmatization: {df['lemmatized_text'].apply(lambda x: len(x.split())).sum()}")

print(f"after removal of stopwords: {df['cleaned_text'].apply(lambda x: len(x.split())).sum()}")

print(f"after stemming and removal of stopwords: {df['cleaned_stemmed_text'].apply(lambda x: len(x.split())).sum()}")

print(f"after lemmatization and removal of stopwords: {df['cleaned_lemmatized_text'].apply(lambda x: len(x.split())).sum()}")

In [ ]:
num_words_train = df['cleaned_lemmatized_text'].apply(lambda x: len(x.split()))
plt.figure(figsize=(12,6))
p1=sns.kdeplot(df['cleaned_lemmatized_text'].apply(lambda x: len(x.split())), fill=True, color="r").set_title('Distribution of Number of Words in Corpus (After Lemmatization and Removal of Stopwords)')

In [ ]:
word_list_train = df.cleaned_lemmatized_text.str.split()
top = Counter([item for sublist in word_list_train for item in sublist])
temp = pd.DataFrame(top.most_common(15))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

In [ ]:
def plot_wordcloud(text, mask=None, max_words=200, max_font_size=100, figure_size=(24.0,16.0), color = 'white',
                   title = None, title_size=40, image_color=False):

    wordcloud = WordCloud(background_color=color,
                    stopwords = STOPWORDS,
                    max_words = max_words,
                    max_font_size = max_font_size, 
                    random_state = 42,
                    width=400, 
                    height=200,
                    mask = mask)
    wordcloud.generate(str(text))
    
    plt.figure(figsize=figure_size)
    if image_color:
        image_colors = ImageColorGenerator(mask)
        plt.imshow(wordcloud.recolor(color_func=image_colors), interpolation="bilinear")
        plt.title(title, fontdict={'size': title_size,  
                                  'verticalalignment': 'bottom'})
    else:
        plt.imshow(wordcloud)
        plt.title(title, fontdict={'size': title_size, 'color': 'black', 
                                  'verticalalignment': 'bottom'})
    plt.axis('off')
    plt.tight_layout()

In [ ]:
pos_mask = np.array(Image.new('RGB', (1000,600)))
plot_wordcloud(word_list_train, mask=pos_mask,color='white',max_font_size=100,title_size=30,title="WordCloud of Corpus")

# **Naive Bayes Classification**

In [ ]:
def naiveBayesClassification(x_train, x_test, y_train, y_test, preprocessing):

    naiveBayes_model = MultinomialNB().fit(x_train, y_train)
    result = naiveBayes_model.predict(x_test)
    confuse = metrics.confusion_matrix(y_test, result)
    
    
    # ==================================================
    # Confusion Matrix
    # ==================================================
    
    fig = plt.figure()
    sns.heatmap(confuse, annot = True, fmt='d')
    
    print("==================================================")
    print("Model: Naive Bayes Classification")
    print("Preprocessing Function: ", preprocessing)
    print("==================================================")
    print()
    plt.title("Confusion matrix of Naive Bayes Classification of Tweets")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    # ==================================================
    # Evaluation Metrics
    # ==================================================
    
    F1_score = metrics.f1_score(y_test, result)
    precision_score = metrics.precision_score(y_test, result)
    recall_score = metrics.recall_score(y_test, result)
    average_precision = metrics.average_precision_score(y_test, result)
    
    print("------------ Evaluation Metrics ----------------")
    print()
    print('F1 score: {0:0.3f}'.format(F1_score))
    print('Precision score: {0:0.3f}'.format(precision_score))
    print('Recall score: {0:0.3f}'.format(recall_score))
    print('Average precision-recall score: {0:0.3f}'.format(average_precision))
    print()

# **K-Nearest Neighbour (KNN) Classification**

In [ ]:
def KNNClassification(x_train, x_test, y_train, y_test, preprocessing):
    
    metric = ['euclidean', 'manhattan', 'cosine']
    neighbors = list(range(1,30))
    leaf_size = list(range(1,50))
    weight = ['uniform', 'distance']
    param_grid = {'metric' : metric, 'n_neighbors': neighbors, 'weights' : weight, 'leaf_size' : leaf_size}
    
    grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=3)
    grid_search.fit(x_train, y_train)
    
    
    knn_model = KNeighborsClassifier(n_neighbors = grid_search.best_params_['n_neighbors'], metric= grid_search.best_params_['metric'], leaf_size = grid_search.best_params_['leaf_size'], weights = grid_search.best_params_['weights'])
    knn_model.fit(x_train, y_train)   
    
    result = knn_model.predict(x_test)
    confuse = metrics.confusion_matrix(y_test, result)
    
    # ==================================================
    # Confusion Matrix
    # ==================================================
    
    fig = plt.figure()
    sns.heatmap(confuse, annot = True, fmt='d')
    
    print("==================================================")
    print("Model: K-Nearest Neighbour Classification")
    print("Best parameters:", grid_search.best_params_)
    print("Preprocessing Function: ", preprocessing)
    print("==================================================")
    print()
    plt.title("Confusion matrix of KNN Classification of tweets")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    
    # ==================================================
    # Evaluation Metrics
    # ==================================================
    
    F1_score = metrics.f1_score(y_test, result)
    precision_score= metrics.precision_score(y_test, result)
    recall_score = metrics.recall_score(y_test, result)
    average_precision = metrics.average_precision_score(y_test, result)

    print("------------ Evaluation Metrics ----------------")
    print()
    print('F1 score: {0:0.3f}'.format(F1_score))
    print('Precision score: {0:0.3f}'.format(precision_score))
    print('Recall score: {0:0.3f}'.format(recall_score))
    print('Average precision-recall score: {0:0.3f}'.format(average_precision))
    print()
  

# **Support Vector Machine (SVM) Classification**

In [ ]:
def SVMClassification(x_train, x_test, y_train, y_test, preprocessing):

    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.05, 0.1, 0.15, 0.20, 0.25]
    degrees = [0, 1, 2, 3, 4, 5, 6]
    kernels = ['rbf', 'linear', 'poly']
    param_grid = {'C': Cs, 'gamma' : gammas, 'degree' : degrees, 'kernel' : kernels}
    
    grid_search = GridSearchCV(svm.SVC(), param_grid, cv = 3)
    grid_search.fit(x_train, y_train)

    SVM_model = svm.SVC(C = grid_search.best_params_['C'], kernel = grid_search.best_params_['kernel'], gamma = grid_search.best_params_['gamma'], degree = grid_search.best_params_['degree'])
    SVM_model.fit(x_train, y_train)
    
    result = SVM_model.predict(x_test)
    confuse = metrics.confusion_matrix(y_test, result)
    
    
    # ==================================================
    # Confusion Matrix
    # ==================================================
    fig = plt.figure()
    sns.heatmap(confuse, annot = True, fmt='d')
    
    print("==================================================")
    print("Model: Support Vector Machine Classification")
    print("Preprocessing Function: ", preprocessing)
    print("Best parameters:", grid_search.best_params_)
    print("==================================================")
    print()
    plt.title("Confusion matrix of SVM Classification of Tweets")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    
    # ==================================================
    # Evaluation Metrics
    # ==================================================
    
    F1_score = metrics.f1_score(y_test, result)
    precision_score= metrics.precision_score(y_test, result)
    recall_score = metrics.recall_score(y_test, result)
    average_precision = metrics.average_precision_score(y_test, result)

    print("------------ Evaluation Metrics ----------------")
    print()
    print('F1 score: {0:0.3f}'.format(F1_score))
    print('Precision score: {0:0.3f}'.format(precision_score))
    print('Recall score: {0:0.3f}'.format(recall_score))
    print('Average precision-recall score: {0:0.3f}'.format(average_precision))
    print()

# **Decision Tree Classification**

In [ ]:
def decisionTreeClassification(x_train, x_test, y_train, y_test, preprocessing):
    
    clf = DecisionTreeClassifier() 

    param_grid = { 
        'criterion': ['gini','entropy'],
        'splitter': ['best','random'],
        'max_features': ['sqrt','log2']
    }
    
    grid_search = GridSearchCV(clf, param_grid, cv= 3)
    grid_search.fit(x_train, y_train)
    grid_search.best_params_

    decisionTree_model = DecisionTreeClassifier(criterion = grid_search.best_params_['criterion'],splitter = grid_search.best_params_['splitter'], max_features = grid_search.best_params_['max_features'])
    decisionTree_model.fit(x_train, y_train)

    result = decisionTree_model.predict(x_test)
    confuse = metrics.confusion_matrix(y_test, result)


    # ==================================================
    # Confusion Matrix
    # ==================================================
    
    fig = plt.figure()
    sns.heatmap(confuse, annot = True, fmt='d')
    
    print("==================================================")
    print("Model: Decision Tree Classification")
    print("Preprocessing Function: ", preprocessing)
    print("Best parameters:", grid_search.best_params_)
    print("==================================================")
    print()
    plt.title("Confusion matrix of Decision Tree Classification of Tweets")
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    # ==================================================
    # Evaluation Metrics
    # ==================================================
    
    F1_score = metrics.f1_score(y_test, result)
    precision_score= metrics.precision_score(y_test, result)
    recall_score = metrics.recall_score(y_test, result)
    average_precision = metrics.average_precision_score(y_test, result)

    print("------------ Evaluation Metrics ----------------")
    print()
    print('F1 score: {0:0.3f}'.format(F1_score))
    print('Precision score: {0:0.3f}'.format(precision_score))
    print('Recall score: {0:0.3f}'.format(recall_score))
    print('Average precision-recall score: {0:0.3f}'.format(average_precision))
    print()

# ***Subjectivity Classification*** 

In [28]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

preprocessings = ['Text', 'stemmed_text', 'cleaned_stemmed_text', 'lemmatized_text', 'cleaned_lemmatized_text', 'cleaned_text'] 
#Count Vectorizer
for preprocessing in preprocessings:
    x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                        df['Subjectivity'],
                                                        test_size = 0.2, 
                                                        random_state = 24)
    
    vectorizer = CountVectorizer(ngram_range=(1,2))
    tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
    test_vector = vectorizer.transform(x_test[preprocessing])
    
    naiveBayesClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    KNNClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    SVMClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    decisionTreeClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)

In [ ]:
#TF-IDF
for preprocessing in preprocessings:
    x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                        df['Subjectivity'],
                                                        test_size = 0.2, 
                                                        random_state = 24)
    vectorizer = TfidfVectorizer(use_idf = True, sublinear_tf = True)
    tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
    test_vector = vectorizer.transform(x_test[preprocessing])
    naiveBayesClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    KNNClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    SVMClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    decisionTreeClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)

In [ ]:
# Best F1 score with cleaned_text & count vectorizer
preprocessing = 'cleaned_text'
x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                    df['Subjectivity'],
                                                    test_size = 0.2, 
                                                    random_state = 24)
    
vectorizer = CountVectorizer(ngram_range=(1,2))
tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
test_vector = vectorizer.transform(x_test[preprocessing])
    
naiveBayesClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)

# Polarity Classification

In [ ]:
df = df[df["Polarity"].notna()]
df.head()

In [ ]:
preprocessings = ['Text', 'stemmed_text', 'cleaned_stemmed_text', 'lemmatized_text', 'cleaned_lemmatized_text', 'cleaned_text'] 
#Count Vectorizer
for preprocessing in preprocessings:
    x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                        df['Polarity'],
                                                        test_size = 0.2, 
                                                        random_state = 24)
    
    vectorizer = CountVectorizer(ngram_range=(1,2))
    tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
    test_vector = vectorizer.transform(x_test[preprocessing])
    
    naiveBayesClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    KNNClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    SVMClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    decisionTreeClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)

In [ ]:
#TF-IDF
for preprocessing in preprocessings:
    x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                        df['Polarity'],
                                                        test_size = 0.2, 
                                                        random_state = 24)
    
    vectorizer = TfidfVectorizer(use_idf = True, sublinear_tf = True)
    tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
    test_vector = vectorizer.transform(x_test[preprocessing])
    
    naiveBayesClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    KNNClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    SVMClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)
    decisionTreeClassification(tweet_matrix, test_vector, y_train, y_test, preprocessing)

In [ ]:
# Best F1 score with cleaned_lemmatized_text & count vectorizer
preprocessing = 'cleaned_lemmatized_text'
x_train, x_test, y_train, y_test = train_test_split(df[[preprocessing]], 
                                                    df['Polarity'],
                                                    test_size = 0.2, 
                                                    random_state = 24)
    
vectorizer = CountVectorizer(ngram_range=(1,2))
tweet_matrix = vectorizer.fit_transform(x_train[preprocessing])
test_vector = vectorizer.transform(x_test[preprocessing])